<img src="../_static/DREGS_logo.png" width="300"/>

# Working with pipeline datasets

This tutorial focuses on how to register data into `DREGS` from a
complete end-to-end pipeline. A "pipeline" in this context is any collection of
datasets that are inter-dependent, i.e., the output data from one process feeds
into the next process as its starting point. For example, a pipeline could
start with some raw imagery from a telescope, this raw imagery is then reduced
and fed into a piece of software that outputs a human-friendly value added
catalog. Or, a pipeline could be from a numerical simulation, starting with the
simulation's initial conditions, which then feed into an N-body code, which
then feed into a structure finder and gets reduced to a halo catalog.

In the DESC data registry nomenclature, each stage of a pipeline is an
"execution", the data product(s) produced during each execution are "datasets",
and executions are linked to one another via "dependencies".

### What we cover in this tutorial

In this tutorial we will learn how to:

- Register a series of dependant datasets from a pipeline into DREGS.

### Before we begin

If you haven't done so already, check out the [getting setup](file:///home/mcalpine/Documents/dataregistry/dataregistry/docs/build/html/tutorial_setup.html) page from the docs if you want to run this tutorial interactively.

A quick way to check everything is set up correctly is to run the first cell below, which should load the `dataregistry` package, and print the package version.

In [1]:
import dataregistry
print("Working with dataregistry version:", dataregistry.__version__)

Working with dataregistry version: 0.2.2


# A pipeline example

For this example, we have a pipeline comprising of three stages. In the first
stage three datasets are produced, one dataset is a directory structure and
the remaining two are individual files. The data output from the first stage
feeds into the second stage as input, which in turn produces its own output (in
this case a directory structure). Finally, the output data from stage two is
fed into the third stage as input and produces its own output dataset directory
structure. Thus our three stages have a simple sequential linking structure;
`Stage1 -> Stage2` and `Stage2 -> Stage3`.

Below is a graphical representation of the setup.

<img src="../_static/pipeline_example.png" width="300"/>

How then would we go about inputting this pipeline into the DESC data registry?

### Connect to DREGS

To begin we establish a link to the data registry using the DREGS class (the Getting Started tutorial goes through this step in more detail).

In [1]:
from dataregistry import DREGS

# Establish connection to database, setting a default owner and owner_type for all registered datasets in this instance.
dregs = DREGS(owner="DESC CO Group", owner_type="group")

OperationalError: (psycopg2.OperationalError) connection to server at "172.17.0.2", port 5432 failed: No route to host
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

Now we can enter our database entries, starting with an execution entry to represent the first stage of our pipeline.

In [4]:
ex1_id = dregs.Registrar.register_execution(
    "pipeline-stage-1",
    description="The first stage of my pipeline",
)

where `ex1_id` is the DREGS index for this execution, which we will reference later.

Next, we register the datasets associated with the output of `pipeline-stage-1`.

In [26]:
dataset_id1 = dregs.Registrar.register_dataset(
    "my-first-pipeline/dataset_1p1/",
    "0.0.1",
    description="A directory structure output from pipeline stage 1",
    old_location="/somewhere/on/machine/my-dataset/",
    execution_id=ex1_id,
    name="Dataset 1.1",
)

dataset_id2 = dregs.Registrar.register_dataset(
    "my-first-pipeline/dataset_1p2.db",
    "0.0.1",
    description="A file output from pipeline stage 1",
    old_location="/somewhere/on/machine/other-datasets/database.db",
    execution_id=ex1_id,
    name="Dataset 1.2",
)

dataset_id3 = dregs.Registrar.register_dataset(
    "my-first-pipeline/dataset_1p3.hdf5",
    "0.0.1",
    description="Another file output from pipeline stage 1",
    old_location="/somewhere/on/machine/other-datasets/info.hdf5",
    execution_id=ex1_id,
    name="Dataset 1.3",
)

Now, the execution for stage two of our pipeline. Note this will automatically generate a dependency between the two executions.

In [2]:
ex2_id = dregs.Registrar.register_execution(
    "pipeline-stage-2",
    description="The second stage of my pipeline",
    input_datasets=[dataset_id1,dataset_id2,dataset_id3],
)

and then to finish, we repeat the process for the remaining datasets and remaining execution.

In [ ]:
dataset_id4 = registrar.register_dataset(
    "my-first-pipeline/dataset_2p1",
    "0.0.1",
    description="A directory structure output from pipeline stage 2",
    old_location="/somewhere/on/machine/my-second-dataset/",
    execution_id=ex2_id,
    name="Dataset 2.1",
)

ex3_id = registrar.register_execution(
    "pipeline-stage-3",
    description="The third stage of my pipeline",
    input_datasets=[dataset_id4],
)

dataset_id5 = registrar.register_dataset(
    "my-first-pipeline/dataset_3p1",
    "0.0.1",
    description="A directory structure output from pipeline stage 3",
    old_location="/somewhere/on/machine/my-third-dataset/",
    execution_id=ex3_id,
    name="Dataset 3.1",
)